In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('data_balita.csv')
df.head()

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm),Status Gizi
0,0,laki-laki,44.591973,stunted
1,0,laki-laki,56.705203,tinggi
2,0,laki-laki,46.863358,normal
3,0,laki-laki,47.508026,normal
4,0,laki-laki,42.743494,severely stunted


In [5]:
df.columns = ['Age(months)', 'Gender', 'Height(cm)', 'Nutrition Status']
df.head()

,Age(months),Gender,Height(cm),Nutrition Status
0,0,laki-laki,44.591973,stunted
1,0,laki-laki,56.705203,tinggi
2,0,laki-laki,46.863358,normal
3,0,laki-laki,47.508026,normal
4,0,laki-laki,42.743494,severely stunted


In [6]:
df['Gender'] = df['Gender'].replace({'laki-laki': 'male', 'perempuan': 'female'})
df.head()

,Age(months),Gender,Height(cm),Nutrition Status
0,0,male,44.591973,stunted
1,0,male,56.705203,tinggi
2,0,male,46.863358,normal
3,0,male,47.508026,normal
4,0,male,42.743494,severely stunted


In [7]:
df['Gender'].value_counts()

Gender
female    61002
male      59997
Name: count, dtype: int64

In [8]:
df['Nutrition Status'].value_counts()

Nutrition Status
normal              67755
severely stunted    19869
tinggi              19560
stunted             13815
Name: count, dtype: int64

In [9]:

df['Nutrition Status'] = df['Nutrition Status'].replace({'tinggi': 'tall'})
df['Nutrition Status'].value_counts()

Nutrition Status
normal              67755
severely stunted    19869
tall                19560
stunted             13815
Name: count, dtype: int64

In [10]:
X = df.drop('Nutrition Status', axis=1)
y = df['Nutrition Status']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)


In [12]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define categorical and numerical features
categorical_features = ['Gender']
numerical_features = ['Age(months)', 'Height(cm)']

# Create transformers
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessor 
pipeline = Pipeline(steps=[('preprocessor', preprocessor)]) 

# Fit and transform the training data (no model defined yet)
X_train_transformed = pipeline.fit_transform(X_train)

# Transform the test data
X_test_transformed = pipeline.transform(X_test)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the KNN model
knn_model = KNeighborsClassifier()

# Create a pipeline with preprocessor and KNN model
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', knn_model)
])

# Fit the pipeline to the training data
knn_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_knn = knn_pipeline.predict(X_test)

# Evaluate the KNN model
accuracy_knn = accuracy_score(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

print("KNN Accuracy:", accuracy_knn)
print("\nKNN Classification Report:\n", report_knn)

KNN Accuracy: 0.9963636363636363

KNN Classification Report:
                   precision    recall  f1-score   support

          normal       1.00      1.00      1.00     13382
severely stunted       1.00      1.00      1.00      4130
         stunted       0.99      0.99      0.99      2790
            tall       0.99      1.00      1.00      3898

        accuracy                           1.00     24200
       macro avg       0.99      1.00      0.99     24200
    weighted avg       1.00      1.00      1.00     24200



In [17]:
import joblib

In [20]:
# Save the model
joblib.dump(knn_pipeline, 'nutrition_model.joblib')

if __name__ == "__main__":
    train_model()

NameError: name 'train_model' is not defined